## CRAWL dari CNBC

### Def untuk crawling dari Portal CNBC
Note : 
- ganti User-Agent pada headers -> ke user agent browser sendiri

In [2]:
import os
import time
import csv
import requests
from bs4 import BeautifulSoup

def republika_batch(keywords, n_berita=200):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
    }

    save_dir = r"E:\$7th\TA\Multimodal_Process_Exploration\DATA\corpus\republika"
    os.makedirs(save_dir, exist_ok=True)

    for keyword in keywords:
        print(f"\n🚀 Mulai scraping: {keyword}")
        data_republika = []
        page = 1

        while len(data_republika) < n_berita:

            alamatURL =  f'https://republika.co.id/search/v3/?q={keyword}&sortby=time&page={page}'
            req = requests.get(alamatURL, headers=headers)
            print(f"🔄 Halaman {page} untuk {keyword}")

            if req.status_code != 200:
                print("❌ Gagal akses halaman.")
                break

            # pakai parser fallback kalau lxml error
            try:
                soup = BeautifulSoup(req.content, 'lxml')
            except Exception:
                soup = BeautifulSoup(req.content, 'html.parser')

            articles = soup.find_all('div', class_='articleItem')
            if not articles:
                print("❌ Tidak ada artikel lagi.")
                break

            for article in articles:
                try:
                    a_tag = article.find('a', href=True)
                    if not a_tag:
                        continue

                    url = a_tag['href']
                    title_tag = article.find('h2', class_='articleTitle')
                    judul = title_tag.get_text(strip=True) if title_tag else a_tag.get_text(strip=True)

                    # buka halaman detail
                    detail_req = requests.get(url, headers=headers)
                    try:
                        detail_soup = BeautifulSoup(detail_req.content, 'lxml')
                    except Exception:
                        detail_soup = BeautifulSoup(detail_req.content, 'html.parser')

                    body = detail_soup.find('div', class_='read__content')
                    berita = body.get_text(separator=' ', strip=True) if body else "Tidak ditemukan"

                    if berita and len(berita) > 30:
                        data_republika.append({
                            'judul': judul,
                            'berita': berita,
                            'url': url,
                        })
                        print(f"[{len(data_republika)}] {judul}")

                        if len(data_republika) >= n_berita:
                            break

                    time.sleep(1)  # delay biar ga ke-block

                except Exception as e:
                    print("⚠ Error ambil artikel:", e)
                    continue

            page += 1

        # Simpan hasil per keyword
        if data_republika:
            filename = os.path.join(save_dir, f"kompas_news_{keyword.lower().replace(' ','')}_{time.strftime('%Y%m%d_%H%M%S')}.csv")
            with open(filename, "w", newline="", encoding="utf-8-sig") as f:
                writer = csv.DictWriter(f, fieldnames=["judul", "berita", "url"])
                writer.writeheader()
                writer.writerows(data_republika)

            print(f"\n✅ {len(data_republika)} artikel '{keyword}' disimpan ke {filename}")
        else:
            print(f"❌ Tidak ada data valid untuk '{keyword}'")



In [3]:
keywords = [
    "BI rate",
    "Pertumbuhan ekonomi",
    "Inflasi",
    "Bank Indonesia",
    "Perbankan digital",
    "IHSG",
    "Pelemahan Rupiah",
    "Investasi",
    "Ekonomi Indonesia",
    "Danantara",
    "OJK",
    "Reksadana",
    "Bursa Efek Indonesia",
    "BEI",
    "Ekonomi digital"
]

republika_batch(keywords, n_berita=10000)



🚀 Mulai scraping: BI rate
🔄 Halaman 1 untuk BI rate
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'BI rate'

🚀 Mulai scraping: Pertumbuhan ekonomi
🔄 Halaman 1 untuk Pertumbuhan ekonomi
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'Pertumbuhan ekonomi'

🚀 Mulai scraping: Inflasi
🔄 Halaman 1 untuk Inflasi
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'Inflasi'

🚀 Mulai scraping: Bank Indonesia
🔄 Halaman 1 untuk Bank Indonesia
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'Bank Indonesia'

🚀 Mulai scraping: Perbankan digital
🔄 Halaman 1 untuk Perbankan digital
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'Perbankan digital'

🚀 Mulai scraping: IHSG
🔄 Halaman 1 untuk IHSG
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'IHSG'

🚀 Mulai scraping: Pelemahan Rupiah
🔄 Halaman 1 untuk Pelemahan Rupiah
❌ Tidak ada artikel lagi.
❌ Tidak ada data valid untuk 'Pelemahan Rupiah'

🚀 Mulai scraping: Investasi
🔄 Halaman 1 untuk Investasi
❌ Tidak ada 